In [1]:
import json
import urllib.request
import time
import datetime
today = datetime.date.today()
import tweepy as tp

In [2]:
gameid = 2020020001
gamedetail = []
starturl = 'https://statsapi.web.nhl.com/api/v1/game/'
endurl = '/feed/live.json'

auth = tp.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tp.API(auth)
time.sleep(1800)

In [3]:
while gameid <= 2020021000:
    pullurl = starturl + str(gameid) + endurl
    with urllib.request.urlopen(pullurl) as url:
        obj = json.loads(url.read().decode())
    allPlays = obj["liveData"]["plays"]['allPlays']
    teamStats = obj["liveData"]["boxscore"]["teams"]
    game_date = obj["gameData"]["datetime"]["dateTime"]
    home_team = obj["gameData"]["teams"]["home"]["name"]
    away_team = obj["gameData"]["teams"]["away"]["name"]
    game_date = game_date.split("T")[0]
    tomorrow = today + datetime.timedelta(days = 1)
    print(f'{gameid}-{obj["gameData"]["status"]["abstractGameState"]}-{game_date}-{tomorrow}')
    # print(obj["gameData"]["status"]["abstractGameState"])
    x=1
    while obj["gameData"]["status"]["abstractGameState"] != "Final" and (home_team == 'Los Angeles Kings' or away_team == 'Los Angeles Kings') and str(game_date) == str(tomorrow):
        print(x)
        x=x+1
        time.sleep(5)
        pullurl = starturl + str(gameid) + endurl
        with urllib.request.urlopen(pullurl) as url:
            obj = json.loads(url.read().decode())
        allPlays = obj["liveData"]["plays"]['allPlays']
        for play in allPlays:
            period = play['about']['period']
            event = play['result']['event']
            timeee = play['about']['periodTime']
            description = play['result']['description']
            if f'Period: {period}-{timeee}, {event}' not in gamedetail:
                gamedetail.append(f'Period: {period}-{timeee}, {event}')
                if event == 'Period Ready' or event == 'Hit' or event == 'Giveaway' or event == 'Takeaway':
                    print(f'NOT TWEETED---Period: {period}-{timeee}, {event}-{description}')
                    #Period Ready
                    #Giveaway
                    #Takeaway
                    #Hit
                elif event == 'Game Scheduled':
                    homescore = teamStats['home']['teamStats']['teamSkaterStats']['goals']
                    awayscore = teamStats['away']['teamStats']['teamSkaterStats']['goals']
                    print(f'Scheduled Game \nScore: {home_team} at {awayscore} #LAKings')
                    api.update_status(f'Scheduled Game \nScore: {home_team} versus {awayscore} #LAKings')
                    time.sleep(8)
                elif event == 'Period Start':
                    #time.sleep(5)
                    #with urllib.request.urlopen(pullurl) as url:
                        #obj = json.loads(url.read().decode())
                    homescore = teamStats['home']['teamStats']['teamSkaterStats']['goals']
                    awayscore = teamStats['away']['teamStats']['teamSkaterStats']['goals']
                    print(f'Score: {home_team}-{homescore}, {away_team}-{awayscore} #LAKings')
                    api.update_status(f'Period Starting \nScore: {home_team}-{homescore}, {away_team}-{awayscore} #LAKings')
                    time.sleep(8)
                    #Game Scheduled
                    #Period Start
                elif event == 'Faceoff' or event == 'Blocked Shot' or event == 'Stoppage' or event == 'Period Official' or event == 'Game End' or event == 'Game Official' or event == 'Shot' or event == 'Penalty' or event == 'Missed Shot' :
                    print(f'Period: {period}-{timeee}, {event}-{description}')
                    api.update_status(f'Period: {period}-{timeee}, {event}-{description} #LAKings')
                    time.sleep(8)
                    #Faceoff
                    #Blocked Shot
                    #Shot
                    #Penalty
                    #Missed Shot
                    #Stoppage
                    #Period Official
                    #Game End
                    #Game Official
                elif event == 'Goal':
                    time.sleep(20)
                    with urllib.request.urlopen(pullurl) as url:
                        obj = json.loads(url.read().decode())
                    homescore = teamStats['home']['teamStats']['teamSkaterStats']['goals']
                    awayscore = teamStats['away']['teamStats']['teamSkaterStats']['goals']
                    api.update_status(f'Period: {period}-{timeee}, Score: {home_team}-{homescore} ~ {away_team}-{awayscore}, {event}-{description}, #LAKings ')
                    #Goal
                elif event == 'Period End' and (period == 1 or period == 2):
                    time.sleep(60)
                    with urllib.request.urlopen(pullurl) as url:
                        obj = json.loads(url.read().decode())
                    homescore = teamStats['home']['teamStats']['teamSkaterStats']['goals']
                    awayscore = teamStats['away']['teamStats']['teamSkaterStats']['goals']
                    print(f'Score: {home_team}-{homescore}, {away_team}-{awayscore} #LAKings')
                    api.update_status(f'Score: {home_team}-{homescore}, {away_team}-{awayscore} #LAKings')
                    time.sleep(900)
                    #Period End
                elif event == 'Period End' and (period != 1 and period != 2):
                    time.sleep(60)
                    with urllib.request.urlopen(pullurl) as url:
                        obj = json.loads(url.read().decode())
                    homescore = teamStats['home']['teamStats']['teamSkaterStats']['goals']
                    awayscore = teamStats['away']['teamStats']['teamSkaterStats']['goals']
                    print(f'Score: {home_team}-{homescore}, {away_team}-{awayscore} #LAKings')
                    api.update_status(f'Score: {home_team}-{homescore}, {away_team}-{awayscore} #LAKings')
                    time.sleep(8)
                else:
                    sleep(0)
    gameid += 1

2020020001-Final-2021-01-13-2021-03-25
2020020002-Final-2021-01-14-2021-03-25
2020020003-Final-2021-01-14-2021-03-25
2020020004-Final-2021-01-14-2021-03-25
2020020005-Final-2021-01-14-2021-03-25
2020020006-Final-2021-01-15-2021-03-25
2020020007-Final-2021-01-15-2021-03-25
2020020008-Final-2021-01-15-2021-03-25
2020020009-Final-2021-02-23-2021-03-25
2020020010-Final-2021-01-15-2021-03-25
2020020011-Final-2021-01-15-2021-03-25
2020020012-Final-2021-01-15-2021-03-25
2020020013-Final-2021-01-15-2021-03-25
2020020014-Final-2021-01-15-2021-03-25
2020020015-Final-2021-01-15-2021-03-25
2020020016-Final-2021-01-15-2021-03-25
2020020017-Final-2021-01-16-2021-03-25
2020020018-Final-2021-01-16-2021-03-25
2020020019-Final-2021-01-16-2021-03-25
2020020020-Preview-2021-05-03-2021-03-25
2020020021-Final-2021-01-16-2021-03-25
2020020022-Final-2021-01-16-2021-03-25
2020020023-Final-2021-01-16-2021-03-25
2020020024-Final-2021-01-17-2021-03-25
2020020025-Final-2021-01-17-2021-03-25
2020020026-Final-2021-0

2020020211-Final-2021-02-10-2021-03-25
2020020212-Final-2021-02-11-2021-03-25
2020020213-Final-2021-02-11-2021-03-25
2020020214-Final-2021-03-15-2021-03-25
2020020215-Final-2021-02-12-2021-03-25
2020020216-Preview-2021-04-25-2021-03-25
2020020217-Final-2021-02-12-2021-03-25
2020020218-Final-2021-02-12-2021-03-25
2020020219-Final-2021-02-12-2021-03-25
2020020220-Final-2021-02-12-2021-03-25
2020020221-Final-2021-02-12-2021-03-25
2020020222-Final-2021-02-12-2021-03-25
2020020223-Preview-2021-03-26-2021-03-25
2020020224-Final-2021-02-12-2021-03-25
2020020225-Preview-2021-04-13-2021-03-25
2020020226-Final-2021-02-12-2021-03-25
2020020227-Final-2021-02-12-2021-03-25
2020020228-Final-2021-02-13-2021-03-25
2020020229-Preview-2021-04-09-2021-03-25
2020020230-Final-2021-02-14-2021-03-25
2020020231-Preview-2021-05-10-2021-03-25
2020020232-Final-2021-02-14-2021-03-25
2020020233-Final-2021-02-14-2021-03-25
2020020234-Final-2021-02-14-2021-03-25
2020020235-Final-2021-02-14-2021-03-25
2020020236-Fina

2020020421-Final-2021-03-12-2021-03-25
2020020422-Final-2021-03-12-2021-03-25
2020020423-Final-2021-03-12-2021-03-25
2020020424-Final-2021-03-12-2021-03-25
2020020425-Final-2021-03-13-2021-03-25
2020020426-Final-2021-03-13-2021-03-25
2020020427-Final-2021-03-13-2021-03-25
2020020428-Final-2021-03-13-2021-03-25
2020020429-Final-2021-03-13-2021-03-25
2020020430-Final-2021-03-13-2021-03-25
2020020431-Final-2021-03-14-2021-03-25
2020020432-Final-2021-03-14-2021-03-25
2020020433-Final-2021-03-14-2021-03-25
2020020434-Final-2021-03-14-2021-03-25
2020020435-Final-2021-03-14-2021-03-25
2020020436-Final-2021-03-13-2021-03-25
2020020437-Final-2021-03-14-2021-03-25
2020020438-Final-2021-03-14-2021-03-25
2020020439-Final-2021-03-14-2021-03-25
2020020440-Final-2021-03-14-2021-03-25
2020020441-Final-2021-03-14-2021-03-25
2020020442-Final-2021-03-14-2021-03-25
2020020443-Final-2021-03-14-2021-03-25
2020020444-Final-2021-03-14-2021-03-25
2020020445-Final-2021-03-14-2021-03-25
2020020446-Final-2021-03-

NameError: name 'awayscore' is not defined

In [ ]:
# gameid = 2020020500

In [ ]:
# gameid = 2020020481
# gamedetail = []
# pullurl = starturl + str(gameid) + endurl
# with urllib.request.urlopen(pullurl) as url:
#     obj = json.loads(url.read().decode())
# allPlays = obj["liveData"]["plays"]['allPlays']
# for play in allPlays:
#     event = play['result']['event']
#     if event not in gamedetail:
#         print(event)
#         gamedetail.append(event)

In [ ]:
# currenttt = time.localtime(time.time() + 24*3600)
# currenttt